In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()

In [3]:
pd.set_option('display.max_columns',0)
pd.set_option('display.max_colwidth',0)

In [4]:
match_data = pd.read_csv('./match_data.csv')
batsman_data = pd.read_csv('./batsman_data.csv')
bowler_data = pd.read_csv('./bowler_data.csv')
df_train = pd.read_csv('./df_train.csv')
df_test = pd.read_csv('./df_test.csv')

In [5]:
df_train['winner_01'] = df_train.apply(lambda x: 0 if (x['team1']==x['winner']) else 1, axis=1)

In [7]:
from sklearn.model_selection import train_test_split
# import xgboost as xgb

In [8]:
match_data.head()

,match id,team1,team2,winner,by,win amount,toss winner,toss decision,venue,city,match_dt,lighting,series_name,season,ground_id,umpire1,umpire2,inning1_runs,inning1_wickets,inning1_balls,inning2_runs,inning2_wickets,inning2_balls,team1_id,team1_roster_ids,team2_id,team2_roster_ids,series_type,winner_id,player_of_the_match_id
0,8638034,Nn Ds,Wn,Wn,wickets,9.0,Wn,field,By Ol,Mount Maunganui,2021-01-01,night match,Sr Sh,2020/21,17681,Dn Mw,TJ Pe,148,6,130,152.0,1.0,97.0,17982,7907451.0:4381761.0:31464.0:258649.0:4949790.0:3834305.0:3776849.0:6718340.0:3566240.0:4950364.0:7543647.0,18570,2653993.0:6718326.0:6718382.0:2486896.0:2288789.0:4950224.0:8187878.0:3083667.0:4950294.0:6718396.0:1585464.0,other_domestic,18570,NaN
1,8588005,Me Rs,Sy Tr,Sy Tr,runs,7.0,Sy Tr,field,Ca Ol,Carrara,2021-01-01,night match,Bg Bh Le,2020/21,6663,D Tr,PJ Ge,166,6,106,117.0,2.0,74.0,33942,37351.0:46794.0:5406540.0:2231928.0:181404.0:1594319.0:7534687.0:7537067.0:1905847.0:8339701.0:7620346.0,33963,1506098.0:1749075.0:36665.0:2083409.0:7534652.0:5788320.0:1612610.0:5509524.0:5497274.0:3876613.0:8193310.0,other_domestic,33963,1749075.0
2,8587837,Sy Ss,Be Ht,Be Ht,wickets,4.0,Sy Ss,bat,Be Ct Gd,Brisbane,2021-01-02,night match,Bg Bh Le,2020/21,1476,GJ Dn,PJ Ge,165,8,126,171.0,6.0,119.0,33956,7869987.0:7620283.0:2076192.0:4002340.0:3063696.0:34061.0:3200756.0:4756982.0:7455818.0:49496.0:1506077.0,33921,7620269.0:2286437.0:87191.0:5786766.0:3114803.0:2535420.0:4967738.0:3995991.0:3890984.0:7353828.0:6818776.0:5419546.0,other_domestic,33921,3890984.0
3,8638041,Nn Ds,Oo,Oo,wickets,2.0,Oo,field,By Ol,Mount Maunganui,2021-01-02,night match,Sr Sh,2020/21,17681,GA St,TJ Pe,153,7,128,156.0,8.0,126.0,17982,7907451.0:4381761.0:31464.0:4949790.0:258649.0:3834305.0:3776849.0:7543647.0:6718340.0:3566240.0:7907458.0,18360,2319638.0:256080.0:7918280.0:3913447.0:2690498.0:6129276.0:2666705.0:6317142.0:3834375.0:2236086.0:4195827.0,other_domestic,18360,NaN
4,8587921,Ht Hs,Me Ss,Ht Hs,runs,21.0,Me Ss,field,Be Ol,Hobart,2021-01-02,day/night match,Bg Bh Le,2020/21,6348,GA Ad,NR Je,164,5,126,143.0,9.0,123.0,33928,4223883.0:2161599.0:1655436.0:5788418.0:319948.0:6249256.0:2340372.0:309056.0:3125849.0:6732004.0:6722540.0,33949,363047.0:2275097.0:3901078.0:2275195.0:4230127.0:2720759.0:2337117.0:6308098.0:7877232.0:3731307.0:4985546.0:2336473.0,other_domestic,33928,3125849.0


In [7]:
from sklearn.model_selection import train_test_split
# import xgboost as xgb

In [112]:
label = df_train['winner_01']

In [113]:
label

0      1
1      0
2      0
3      0
4      1
      ..
943    0
944    1
945    1
946    1
947    1
Name: winner_01, Length: 948, dtype: int64

In [114]:
df_train.head()

,match id,team1,team1_id,team1_roster_ids,team2,team2_id,team2_roster_ids,winner,winner_id,toss winner,toss decision,venue,city,match_dt,lighting,series_name,season,ground_id,team_count_50runs_last15,team_winp_last5,team1only_avg_runs_last15,team1_winp_team2_last15,ground_avg_runs_last15,winner_01
0,9331181,Ba,11283,9373356.0:7857520.0:4232164.0:4566540.0:3299407.0:232000.0:7883504.0:8725723.0:7878989.0:2526390.0:8725730.0:4017523.0,Hl Ph,12634,3500958.0:4231751.0:2735081.0:2035102.0:3698337.0:5058192.0:6844718.0:4566526.0:6596582.0:4161975.0:7952804.0:8465057.0,Hl Ph,12634,Hl Ph,field,Hr Ct Sm Ie,Indore,2022-10-20,day/night match,Sd Mq Ai Ty,2022/23,7398,1.666667,0.672131,139.000000,100.0,157.178571,1
1,8797060,Ed,20,2089079.0:6139370.0:2076192.0:62432.0:2083409.0:172199.0:2022957.0:1482998.0:1711492.0:4507208.0:3231437.0,Wt Is,41,4690258.0:4069666.0:4230127.0:1942317.0:1613926.0:2740408.0:4739552.0:5744850.0:4690104.0:3715697.0:3468870.0,Ed,20,Wt Is,field,Kn Ol Bn Bs,Bridgetown,2022-01-23,day/night match,Ed tr of Wt Is,2021/22,1406,1.285714,1.952381,156.000000,50.0,103.500000,0
2,9433269,We,10576,3298427.0:2288789.0:7773338.0:3519011.0:3681957.0:7671313.0:5320118.0:2922611.0:7184232.0:7661996.0:6240632.0,Ne,8987,4003390.0:1749075.0:1626526.0:4172447.0:5516720.0:7328110.0:125915.0:130437.0:7507303.0:4420828.0:5652828.0,We,10576,Ne,field,Tt Be Nm,Nottingham,2023-06-02,day/night match,Vy Bt,2023,251,0.857143,0.672131,173.266667,0.0,154.333333,0
3,9587073,Ga An Ws,36084,8127230.0:4690328.0:4069666.0:7960847.0:4690188.0:4739580.0:4739552.0:5744850.0:2294823.0:9266102.0:284339.0,Bs Rs,36070,3462080.0:2436405.0:1798705.0:7550857.0:5742470.0:2740408.0:3575774.0:3200756.0:334963.0:6399210.0:6347494.0,Ga An Ws,36084,Ga An Ws,bat,Bn La Sm Ta Td,Tarouba,2023-09-10,day match,Cn Pr Le,2023,14300,2.166667,1.975610,164.266667,50.0,144.250000,0
4,9516457,Pb Ks,30407,8127181.0:197658.0:4239038.0:2398346.0:5053082.0:4640824.0:5038046.0:8180500.0:2035102.0:3851518.0:7881845.0:7453697.0,Gt Ts,48341,1958683.0:7491224.0:8059029.0:4377610.0:2252452.0:2966879.0:5554254.0:4690230.0:3373285.0:3759846.0:6496482.0,Gt Ts,48341,Gt Ts,field,Pb Ct An IS Ba Sm Mi Ch,Chandigarh,2023-04-13,night match,In Pr Le,2023,7118,0.818182,1.327869,164.666667,0.0,189.000000,1


In [115]:
batsman_data.head()

,match id,batsman,batsman_id,batsman_details,is_batsman_captain,is_batsman_keeper,inning,runs,balls_faced,over_faced_first,wicket kind,out_by_bowler,out_by_fielder,bowler_id,bowler_details,is_bowler_keeper,is_bowler_captain,strike_rate,Fours,Sixes,match_dt
0,8638034,KD Ce,7907451.0,NZ:Right-hand bat:Right-arm medium-fast:,0.0,0.0,1,7,5,1.1,caught,JS Nm,TM Jn,2486896.0,NZ:Left-hand bat:Right-arm medium-fast:,0.0,0.0,140.00,1.0,NaN,2021-01-01
1,8638034,TL St,4381761.0,NZ:Right-hand bat:None:,0.0,1.0,1,46,46,1.2,caught,R Ra,MG Bl,6718382.0,NZ:Left-hand bat:Slow left-arm orthodox:,0.0,0.0,100.00,4.0,1.0,2021-01-01
2,8638034,HR Cr,4949790.0,NZ:Right-hand bat:Right-arm offbreak:,0.0,0.0,1,9,10,12.1,caught,PF Yd,R Ra,4950294.0,NZ:Right-hand bat:Legbreak googly:,0.0,0.0,90.00,NaN,NaN,2021-01-01
3,8638034,BR Hn,3834305.0,NZ:Right-hand bat:Right-arm medium-fast:,0.0,0.0,1,28,22,13.6,caught,HK Bt,FH An,1585464.0,NZ:Left-hand bat:Right-arm medium-fast:,0.0,0.0,127.27,3.0,NaN,2021-01-01
4,8638034,SC Kn,3776849.0,NZ:Right-hand bat:Right-arm fast-medium:,0.0,0.0,1,18,13,17.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138.46,NaN,1.0,2021-01-01


In [116]:
avg_runs_per_inning = batsman_data.groupby('batsman_id')['runs'].mean().reset_index()
avg_runs_per_inning.rename(columns={'runs': 'avg_runs_per_inning'}, inplace=True)

In [117]:
avg_runs_per_inning

,batsman_id,avg_runs_per_inning
0,31464.0,3.333333
1,34061.0,11.805556
2,36665.0,20.800000
3,37351.0,20.722222
4,41740.0,22.310345
...,...,...
1630,9663968.0,17.666667
1631,9822070.0,7.000000
1632,9822077.0,0.000000
1633,9822140.0,7.000000


In [118]:
batsman_stats = batsman_data.groupby('batsman_id').agg({'runs': 'sum', 'balls_faced': 'sum'}).reset_index()

# Calculate runs per ball faced
batsman_stats['runs_per_ball_faced'] = batsman_stats['runs'] / batsman_stats['balls_faced']

In [119]:
batsman_stats

,batsman_id,runs,balls_faced,runs_per_ball_faced
0,31464.0,10,18,0.555556
1,34061.0,425,297,1.430976
2,36665.0,208,171,1.216374
3,37351.0,746,598,1.247492
4,41740.0,647,496,1.304435
...,...,...,...,...
1630,9663968.0,53,41,1.292683
1631,9822070.0,7,21,0.333333
1632,9822077.0,0,2,0.000000
1633,9822140.0,7,21,0.333333


In [120]:
batsman1 = batsman_stats.merge( avg_runs_per_inning,on = 'batsman_id')

In [121]:
batsman1

,batsman_id,runs,balls_faced,runs_per_ball_faced,avg_runs_per_inning
0,31464.0,10,18,0.555556,3.333333
1,34061.0,425,297,1.430976,11.805556
2,36665.0,208,171,1.216374,20.800000
3,37351.0,746,598,1.247492,20.722222
4,41740.0,647,496,1.304435,22.310345
...,...,...,...,...,...
1630,9663968.0,53,41,1.292683,17.666667
1631,9822070.0,7,21,0.333333,7.000000
1632,9822077.0,0,2,0.000000,0.000000
1633,9822140.0,7,21,0.333333,7.000000


In [122]:
# def categorize_runs_per_ball(runs_per_ball):
#     if runs_per_ball < 1:
#         return 0
#     elif runs_per_ball < 1.2:
#         return 2
#     elif runs_per_ball < 1.4:
#         return 3
#     else:
#         return 5
    
# def categorize_avg_runs(avg_runs):
#     if avg_runs < 10:
#         return 0
#     elif avg_runs < 20:
#         return 2
#     elif avg_runs < 30:
#         return 3
#     else:
#         return 5

# # Apply the function to create the 'rating' column
# #batsman_data['rating'] = batsman_data['avg_runs_per_inning'].apply(categorize_avg_runs)

# # Apply the function to create a new column
# #batsman_stats['rating'] = batsman_stats['runs_per_ball_faced'].apply(categorize_runs_per_ball)
# batsman1['rating2'] = batsman1['avg_runs_per_inning'].apply(categorize_avg_runs)

# Display the result
batsman1['total'] = batsman1['avg_runs_per_inning']* batsman1['runs_per_ball_faced']

In [124]:
#batsman1

In [125]:
batsman_rating = {}

In [126]:
for index, row in batsman1.iterrows():
    rating = row['total']
    batsman_id = row['batsman_id']
    batsman_rating[batsman_id] = rating

In [128]:
#batsman_rating

In [130]:
bowler_data.head()

,match id,bowler,bowler_id,bowler_details,is_bowler_captain,is_bowler_keeper,inning,runs,wicket_count,balls_bowled,economy,maiden,dots,Fours,Sixes,wides,noballs,match_dt
0,8638034,BV Ss,6718396.0,NZ:Right-hand bat:Right-arm medium-fast:,0.0,0.0,1,21,0,12,10.50,0,3.0,1,1,3,1,2021-01-01
1,8638034,HK Bt,1585464.0,NZ:Left-hand bat:Right-arm medium-fast:,0.0,0.0,1,34,1,24,8.50,0,7.0,3,1,2,0,2021-01-01
2,8638034,JS Nm,2486896.0,NZ:Left-hand bat:Right-arm medium-fast:,0.0,0.0,1,23,2,24,5.75,0,12.0,1,1,0,0,2021-01-01
3,8638034,LV vn Bk,3083667.0,NED:Right-hand bat:Right-arm medium-fast:,0.0,0.0,1,31,1,24,7.75,0,9.0,4,0,2,0,2021-01-01
4,8638034,PF Yd,4950294.0,NZ:Right-hand bat:Legbreak googly:,0.0,0.0,1,21,1,24,5.25,0,8.0,0,0,1,0,2021-01-01


In [128]:
#batsman_rating

In [128]:
#batsman_rating

In [128]:
#batsman_rating

In [129]:
#df_train['team1'].unique()

In [12]:
from sklearn.preprocessing import LabelEncoder

In [14]:
from sklearn import preprocessing

In [16]:
le = preprocessing.LabelEncoder()
df_train['team1'] = le.fit_transform(df_train.team1.values)

In [18]:
df_train['team2'] = le.fit_transform(df_train.team2.values)

In [20]:
df_test['team2'] = le.fit_transform(df_test.team2.values)
df_test['team1'] = le.fit_transform(df_test.team1.values)

In [35]:
df_train['toss_winner'] = df_train['toss winner']

In [36]:
df_train.head()

,match id,team1,team1_id,team1_roster_ids,team2,team2_id,team2_roster_ids,winner,winner_id,toss winner,toss decision,venue,city,match_dt,lighting,series_name,season,ground_id,team_count_50runs_last15,team_winp_last5,team1only_avg_runs_last15,team1_winp_team2_last15,ground_avg_runs_last15,winner_01,toss_winner
0,9331181,6,11283,9373356.0:7857520.0:4232164.0:4566540.0:3299407.0:232000.0:7883504.0:8725723.0:7878989.0:2526390.0:8725730.0:4017523.0,43,12634,3500958.0:4231751.0:2735081.0:2035102.0:3698337.0:5058192.0:6844718.0:4566526.0:6596582.0:4161975.0:7952804.0:8465057.0,43,12634,Hl Ph,field,Hr Ct Sm Ie,Indore,2022-10-20,day/night match,Sd Mq Ai Ty,2022/23,7398,1.666667,0.672131,139.000000,100.0,157.178571,1,Hl Ph
1,8797060,35,20,2089079.0:6139370.0:2076192.0:62432.0:2083409.0:172199.0:2022957.0:1482998.0:1711492.0:4507208.0:3231437.0,138,41,4690258.0:4069666.0:4230127.0:1942317.0:1613926.0:2740408.0:4739552.0:5744850.0:4690104.0:3715697.0:3468870.0,28,20,Wt Is,field,Kn Ol Bn Bs,Bridgetown,2022-01-23,day/night match,Ed tr of Wt Is,2021/22,1406,1.285714,1.952381,156.000000,50.0,103.500000,0,Wt Is
2,9433269,135,10576,3298427.0:2288789.0:7773338.0:3519011.0:3681957.0:7671313.0:5320118.0:2922611.0:7184232.0:7661996.0:6240632.0,81,8987,4003390.0:1749075.0:1626526.0:4172447.0:5516720.0:7328110.0:125915.0:130437.0:7507303.0:4420828.0:5652828.0,125,10576,Ne,field,Tt Be Nm,Nottingham,2023-06-02,day/night match,Vy Bt,2023,251,0.857143,0.672131,173.266667,0.0,154.333333,0,Ne
3,9587073,39,36084,8127230.0:4690328.0:4069666.0:7960847.0:4690188.0:4739580.0:4739552.0:5744850.0:2294823.0:9266102.0:284339.0,13,36070,3462080.0:2436405.0:1798705.0:7550857.0:5742470.0:2740408.0:3575774.0:3200756.0:334963.0:6399210.0:6347494.0,32,36084,Ga An Ws,bat,Bn La Sm Ta Td,Tarouba,2023-09-10,day match,Cn Pr Le,2023,14300,2.166667,1.975610,164.266667,50.0,144.250000,0,Ga An Ws
4,9516457,96,30407,8127181.0:197658.0:4239038.0:2398346.0:5053082.0:4640824.0:5038046.0:8180500.0:2035102.0:3851518.0:7881845.0:7453697.0,38,48341,1958683.0:7491224.0:8059029.0:4377610.0:2252452.0:2966879.0:5554254.0:4690230.0:3373285.0:3759846.0:6496482.0,38,48341,Gt Ts,field,Pb Ct An IS Ba Sm Mi Ch,Chandigarh,2023-04-13,night match,In Pr Le,2023,7118,0.818182,1.327869,164.666667,0.0,189.000000,1,Gt Ts


In [31]:
df_train['winner'] = le.fit_transform(df_train.winner.values)


In [42]:
df_train['toss_winner'] = le.fit_transform(df_train.toss_winner.values)
df_test['toss_winner'] = le.fit_transform(df_test.toss_winner.values)

In [43]:
df_train.head()


,match id,team1,team1_id,team1_roster_ids,team2,team2_id,team2_roster_ids,winner,winner_id,toss winner,toss decision,venue,city,match_dt,lighting,series_name,season,ground_id,team_count_50runs_last15,team_winp_last5,team1only_avg_runs_last15,team1_winp_team2_last15,ground_avg_runs_last15,winner_01,toss_winner
0,9331181,6,11283,9373356.0:7857520.0:4232164.0:4566540.0:3299407.0:232000.0:7883504.0:8725723.0:7878989.0:2526390.0:8725730.0:4017523.0,43,12634,3500958.0:4231751.0:2735081.0:2035102.0:3698337.0:5058192.0:6844718.0:4566526.0:6596582.0:4161975.0:7952804.0:8465057.0,43,12634,44,field,Hr Ct Sm Ie,Indore,2022-10-20,day/night match,Sd Mq Ai Ty,2022/23,7398,1.666667,0.672131,139.000000,100.0,157.178571,1,44
1,8797060,35,20,2089079.0:6139370.0:2076192.0:62432.0:2083409.0:172199.0:2022957.0:1482998.0:1711492.0:4507208.0:3231437.0,138,41,4690258.0:4069666.0:4230127.0:1942317.0:1613926.0:2740408.0:4739552.0:5744850.0:4690104.0:3715697.0:3468870.0,28,20,138,field,Kn Ol Bn Bs,Bridgetown,2022-01-23,day/night match,Ed tr of Wt Is,2021/22,1406,1.285714,1.952381,156.000000,50.0,103.500000,0,138
2,9433269,135,10576,3298427.0:2288789.0:7773338.0:3519011.0:3681957.0:7671313.0:5320118.0:2922611.0:7184232.0:7661996.0:6240632.0,81,8987,4003390.0:1749075.0:1626526.0:4172447.0:5516720.0:7328110.0:125915.0:130437.0:7507303.0:4420828.0:5652828.0,125,10576,81,field,Tt Be Nm,Nottingham,2023-06-02,day/night match,Vy Bt,2023,251,0.857143,0.672131,173.266667,0.0,154.333333,0,81
3,9587073,39,36084,8127230.0:4690328.0:4069666.0:7960847.0:4690188.0:4739580.0:4739552.0:5744850.0:2294823.0:9266102.0:284339.0,13,36070,3462080.0:2436405.0:1798705.0:7550857.0:5742470.0:2740408.0:3575774.0:3200756.0:334963.0:6399210.0:6347494.0,32,36084,34,bat,Bn La Sm Ta Td,Tarouba,2023-09-10,day match,Cn Pr Le,2023,14300,2.166667,1.975610,164.266667,50.0,144.250000,0,34
4,9516457,96,30407,8127181.0:197658.0:4239038.0:2398346.0:5053082.0:4640824.0:5038046.0:8180500.0:2035102.0:3851518.0:7881845.0:7453697.0,38,48341,1958683.0:7491224.0:8059029.0:4377610.0:2252452.0:2966879.0:5554254.0:4690230.0:3373285.0:3759846.0:6496482.0,38,48341,40,field,Pb Ct An IS Ba Sm Mi Ch,Chandigarh,2023-04-13,night match,In Pr Le,2023,7118,0.818182,1.327869,164.666667,0.0,189.000000,1,40


In [44]:
df_test['toss_winner'] = df_test['toss winner']

In [48]:
df_train['toss_decision'] = df_train['toss decision']
df_test['toss_decision'] = df_test['toss decision']

df_train['toss_decision'] = le.fit_transform(df_train.toss_decision.values)
df_test['toss_decision'] = le.fit_transform(df_test.toss_decision.values)

In [49]:
df_train.head()

,match id,team1,team1_id,team1_roster_ids,team2,team2_id,team2_roster_ids,winner,winner_id,toss winner,toss decision,venue,city,match_dt,lighting,series_name,season,ground_id,team_count_50runs_last15,team_winp_last5,team1only_avg_runs_last15,team1_winp_team2_last15,ground_avg_runs_last15,winner_01,toss_winner,toss_decision
0,9331181,6,11283,9373356.0:7857520.0:4232164.0:4566540.0:3299407.0:232000.0:7883504.0:8725723.0:7878989.0:2526390.0:8725730.0:4017523.0,43,12634,3500958.0:4231751.0:2735081.0:2035102.0:3698337.0:5058192.0:6844718.0:4566526.0:6596582.0:4161975.0:7952804.0:8465057.0,43,12634,44,field,Hr Ct Sm Ie,Indore,2022-10-20,day/night match,Sd Mq Ai Ty,2022/23,7398,1.666667,0.672131,139.000000,100.0,157.178571,1,44,1
1,8797060,35,20,2089079.0:6139370.0:2076192.0:62432.0:2083409.0:172199.0:2022957.0:1482998.0:1711492.0:4507208.0:3231437.0,138,41,4690258.0:4069666.0:4230127.0:1942317.0:1613926.0:2740408.0:4739552.0:5744850.0:4690104.0:3715697.0:3468870.0,28,20,138,field,Kn Ol Bn Bs,Bridgetown,2022-01-23,day/night match,Ed tr of Wt Is,2021/22,1406,1.285714,1.952381,156.000000,50.0,103.500000,0,138,1
2,9433269,135,10576,3298427.0:2288789.0:7773338.0:3519011.0:3681957.0:7671313.0:5320118.0:2922611.0:7184232.0:7661996.0:6240632.0,81,8987,4003390.0:1749075.0:1626526.0:4172447.0:5516720.0:7328110.0:125915.0:130437.0:7507303.0:4420828.0:5652828.0,125,10576,81,field,Tt Be Nm,Nottingham,2023-06-02,day/night match,Vy Bt,2023,251,0.857143,0.672131,173.266667,0.0,154.333333,0,81,1
3,9587073,39,36084,8127230.0:4690328.0:4069666.0:7960847.0:4690188.0:4739580.0:4739552.0:5744850.0:2294823.0:9266102.0:284339.0,13,36070,3462080.0:2436405.0:1798705.0:7550857.0:5742470.0:2740408.0:3575774.0:3200756.0:334963.0:6399210.0:6347494.0,32,36084,34,bat,Bn La Sm Ta Td,Tarouba,2023-09-10,day match,Cn Pr Le,2023,14300,2.166667,1.975610,164.266667,50.0,144.250000,0,34,0
4,9516457,96,30407,8127181.0:197658.0:4239038.0:2398346.0:5053082.0:4640824.0:5038046.0:8180500.0:2035102.0:3851518.0:7881845.0:7453697.0,38,48341,1958683.0:7491224.0:8059029.0:4377610.0:2252452.0:2966879.0:5554254.0:4690230.0:3373285.0:3759846.0:6496482.0,38,48341,40,field,Pb Ct An IS Ba Sm Mi Ch,Chandigarh,2023-04-13,night match,In Pr Le,2023,7118,0.818182,1.327869,164.666667,0.0,189.000000,1,40,1


In [60]:
features = df_train[['team1', 'team2', 'toss_winner','toss_decision','ground_id', 'team_count_50runs_last15',
                       'team_winp_last5', 'team1only_avg_runs_last15', 'ground_avg_runs_last15']]
target = df_train['winner_01']


In [61]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 948 entries, 0 to 947
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   team1                      948 non-null    int64  
 1   team2                      948 non-null    int64  
 2   toss_winner                948 non-null    int64  
 3   toss_decision              948 non-null    int64  
 4   ground_id                  948 non-null    int64  
 5   team_count_50runs_last15   948 non-null    float64
 6   team_winp_last5            948 non-null    float64
 7   team1only_avg_runs_last15  927 non-null    float64
 8   ground_avg_runs_last15     895 non-null    float64
dtypes: float64(4), int64(5)
memory usage: 66.8 KB


In [50]:
# Selecting features and target for the model
# features = df_train[['team1', 'team1_id', 'team1_roster_ids', 'team2', 'team2_id', 'team2_roster_ids', 'toss winner', 'toss decision', 'venue', 'team_count_50runs_last15',
#                       'team_winp_last5', 'team1only_avg_runs_last15', 'ground_avg_runs_last15']]
# target = train_data['winner_01']

# # Convert categorical variables to dummy variables
# #features_encoded = pd.get_dummies(features)

# features_test = test_data[['team1', 'team1_id', 'team1_roster_ids', 'team2', 'team2_id', 'team2_roster_ids', 'toss winner', 'toss decision', 'venue', 'team_count_50runs_last15',
#                       'team_winp_last5', 'team1only_avg_runs_last15', 'ground_avg_runs_last15']]
# #target_test = test_data['winner_01']

# # Convert categorical variables to dummy variables
# features_encoded_test = pd.get_dummies(features_test)


In [62]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.2, random_state=42)

In [63]:
# Initialize and train the XGBoost model
model = xgb.XGBClassifier(objective='multi:softmax', num_class=2)
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None, num_class=2,
              num_parallel_tree=None, ...)

In [64]:
predictions = model.predict(X_test)

In [65]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,predictions)

0.5105263157894737

In [67]:
#predictions_test_ = model.predict(embeddings_test)